In [1]:
import pandas as pd
import numpy as np
import talib as ta
from pycoingecko import CoinGeckoAPI
from datetime import datetime as dt, timedelta as td

In [2]:
config = {
    'id': 'bitcoin',
    'vs_curr': 'usd',
    'range': 40, # max 24
    'save_csv_path': 'datasets/crypto_ta_btc_04.csv'
}

In [3]:
now = dt.strptime(dt.now().strftime("%Y-%m-%d %H:00:00"),"%Y-%m-%d %H:00:00")
timestamp_list = [{'from': dt.timestamp(now - td(days=(x+1)*60)), 'to': dt.timestamp(now - td(days=x*60))} for x in reversed(range(config['range']))]

print(timestamp_list)

[{'from': 1428447600.0, 'to': 1433631600.0}, {'from': 1433631600.0, 'to': 1438815600.0}, {'from': 1438815600.0, 'to': 1443999600.0}, {'from': 1443999600.0, 'to': 1449187200.0}, {'from': 1449187200.0, 'to': 1454371200.0}, {'from': 1454371200.0, 'to': 1459551600.0}, {'from': 1459551600.0, 'to': 1464735600.0}, {'from': 1464735600.0, 'to': 1469919600.0}, {'from': 1469919600.0, 'to': 1475103600.0}, {'from': 1475103600.0, 'to': 1480291200.0}, {'from': 1480291200.0, 'to': 1485475200.0}, {'from': 1485475200.0, 'to': 1490655600.0}, {'from': 1490655600.0, 'to': 1495839600.0}, {'from': 1495839600.0, 'to': 1501023600.0}, {'from': 1501023600.0, 'to': 1506207600.0}, {'from': 1506207600.0, 'to': 1511395200.0}, {'from': 1511395200.0, 'to': 1516579200.0}, {'from': 1516579200.0, 'to': 1521763200.0}, {'from': 1521763200.0, 'to': 1526943600.0}, {'from': 1526943600.0, 'to': 1532127600.0}, {'from': 1532127600.0, 'to': 1537311600.0}, {'from': 1537311600.0, 'to': 1542499200.0}, {'from': 1542499200.0, 'to': 15

In [4]:
cg = CoinGeckoAPI()
temp_closes = []
temp_volumes = []
temp_timestamp = []

# bitcoin_close = pd.read_csv('bitcoin_close_full.csv')

for timestamp in timestamp_list:
    mk_chart = cg.get_coin_market_chart_range_by_id(id=config['id'], vs_currency=config['vs_curr'], from_timestamp=timestamp['from'], to_timestamp=timestamp['to'])
    for i in range(len(mk_chart['prices'])):
        temp_closes.append(mk_chart['prices'][i][1])
        temp_timestamp.append(dt.fromtimestamp(mk_chart['prices'][i][0]/1000))
        temp_volumes.append(mk_chart['total_volumes'][i][1])

closes = np.array(temp_closes)
dates = np.array(temp_timestamp)
volumes = np.array(temp_volumes)
# closes = np.array(bitcoin_close['closes'])
# volumes = np.array(bitcoin_close['volumes'])

print(closes)
print(volumes)

[  244.2709       243.20065485   234.5253     ... 60767.15503357
 60950.81919978 61185.32028944]
[1.06548662e+08 9.37494352e+07 1.10710603e+08 ... 3.63370907e+10
 3.63159899e+10 3.61392227e+10]


In [5]:
sma_200 = ta.ROC(ta.SMA(closes, 200), timeperiod=1)
ema_8 = ta.ROC(ta.EMA(closes, 8), timeperiod=1)
ema_20 = ta.ROC(ta.EMA(closes, 20), timeperiod=1)
ema_25 = ta.ROC(ta.EMA(closes, 25), timeperiod=1)
ema_30 = ta.ROC(ta.EMA(closes, 30), timeperiod=1)
ema_35 = ta.ROC(ta.EMA(closes, 35), timeperiod=1)
ema_40 = ta.ROC(ta.EMA(closes, 40), timeperiod=1)
ema_45 = ta.ROC(ta.EMA(closes, 45), timeperiod=1)
ema_50 = ta.ROC(ta.EMA(closes, 50), timeperiod=1)
sma_vol = ta.ROC(ta.SMA(volumes, 20), timeperiod=1)
# bbands_up, bbands_mid, bbands_low = ta.BBANDS(closes, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
roc = ta.ROC(closes, timeperiod=1)
cmo = ta.CMO(closes, timeperiod=14)
macd, macdsignal, macdhist = ta.MACD(closes, fastperiod=12, slowperiod=26, signalperiod=9)
macdhist = ta.ROC(macdhist, timeperiod=1)
ppo = ta.PPO(closes, fastperiod=12, slowperiod=26, matype=0)
linearreg = ta.LINEARREG(closes, timeperiod=14)
tsf = ta.TSF(closes, timeperiod=14)

In [6]:
class_data = []
roc = ta.ROC(closes, timeperiod=1)

for n in range(0,len(roc)):
    if n + 5 < len(roc):
        if roc[n] < 0 and roc[n+1] < 0 and roc[n+2] < 0 and roc[n+3] < 0 and roc[n+4] < 0 and roc[n+5]:
            class_data.append('VLOW')
        elif roc[n] < 0 and roc[n+1] < 0 and roc[n+2] < 0:
            class_data.append('LOW')
        elif roc[n] > 0 and roc[n+1] > 0 and roc[n+2] > 0 and roc[n+3] > 0 and roc[n+4] > 0 and roc[n+5]:
            class_data.append('VHIGH')
        elif roc[n] > 0 and roc[n+1] > 0 and roc[n+2] > 0:
            class_data.append('HIGH')
        else:
            class_data.append('NEUTRAL')
            
print(len(class_data))

30081


In [7]:
dt = pd.DataFrame({
    'sma_200': sma_200[200:-6],
    'ema_8': ema_8[200:-6],
    'ema_20': ema_20[200:-6],
    'ema_25': ema_25[200:-6],
    'ema_30': ema_30[200:-6],
    'ema_35': ema_35[200:-6],
    'ema_40': ema_40[200:-6],
    'ema_45': ema_45[200:-6],
    'ema_50': ema_50[200:-6],
    'sma_vol': sma_vol[200:-6],
    'roc': roc[200:-6],
    'cmo': cmo[200:-6],
    'macd': macd[200:-6],
    'macdsignal': macdsignal[200:-6],
    'macdhist': macdhist[200:-6],
    'ppo': ppo[200:-6],
    'linearreg': linearreg[200:-6],
    'tsf': tsf[200:-6],
    'trend': np.array(class_data[201::]) # tirado 5 valores do final
})

dt.describe()
print(dt)

        sma_200     ema_8    ema_20    ema_25    ema_30    ema_35    ema_40  \
0      0.082121  1.113726  0.898654  0.822114  0.752451  0.689303  0.632477   
1      0.085778  0.918847  0.833647  0.775431  0.717906  0.663256  0.612591   
2      0.122098  1.454532  1.082362  0.983777  0.898506  0.823368  0.756792   
3      0.135912  1.813851  1.282965  1.156504  1.051171  0.960714  0.881920   
4      0.157913  2.184190  1.506989  1.351232  1.224271  1.117026  1.024697   
...         ...       ...       ...       ...       ...       ...       ...   
29875  0.005357  0.076296  0.082020  0.070789  0.060964  0.052944  0.046534   
29876  0.006016  0.085184  0.085300  0.074311  0.064558  0.056489  0.049963   
29877  0.003267 -0.117699 -0.002105  0.004511  0.006625  0.007043  0.006862   
29878  0.004103 -0.056183  0.013356  0.016498  0.016547  0.015566  0.014355   
29879  0.002314 -0.190540 -0.051043 -0.035796 -0.027334 -0.022175 -0.018729   

         ema_45    ema_50   sma_vol       roc      

In [8]:
# dt.to_csv(config['save_csv_path'], index=False)

In [9]:
orig_dt = pd.DataFrame({'closes': closes, 'dates': dates, 'volumes': volumes})
orig_dt.to_csv('bitcoin_close_full.csv', index=False)